In [1]:
import findspark
findspark.init()     

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext, SQLContext
from pyspark.streaming import StreamingContext
from pyspark.sql.types import * 
from pyspark.sql.functions import *
import socket
import json



spark = SparkSession \
    .builder \
    .config("spark.jars", "/home/kini/Downloads/postgresql-42.3.3.jar") \
    .master("local[2]") \
    .appName("app_hack") \
    .enableHiveSupport() \
    .getOrCreate()


sqlContext = SQLContext(spark.sparkContext)   
ssc = StreamingContext(spark.sparkContext,30)
lines = ssc.socketTextStream('localhost', 9002)

def saveRDD(rdd):
                             
    if not rdd.isEmpty():
        
        validJsonRdd = rdd.map(lambda x : x.replace(",1],", ",").replace("}]", "}"))
        df = sqlContext.read.json(validJsonRdd)
        df = df.withColumn("current_timestamp", current_timestamp())
        
        
        user_postgres = "postgres"
        password_postgres = 
        
        df.write \
        .mode("append") \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://172.20.0.2:5432/db_accident") \
        .option("dbtable", "Currency") \
        .option("user", user_postgres) \
        .option("password", password_postgres) \
        .option("driver", "org.postgresql.Driver") \
        .save()
        
        
        df.write.option("header",True) \
        .partitionBy("current_timestamp") \
        .mode("overwrite") \
        .csv("/Cur_csv")
    

lines.foreachRDD(lambda rdd : saveRDD(rdd))


ssc.start()
#ssc.awaitTermination()

22/02/20 10:27:10 WARN Utils: Your hostname, kini-G3-3579 resolves to a loopback address: 127.0.1.1; using 192.168.0.100 instead (on interface wlo1)
22/02/20 10:27:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/20 10:27:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/opt/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
